In [2]:
%load_ext autoreload

%autoreload 2
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn import metrics

from mlxtend.plotting import plot_decision_regions
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from ast import literal_eval

import warnings
import numpy as np
from collections import OrderedDict

from lob_data_utils import lob, db_result, model
from lob_data_utils.svm_calculation import lob_svm
import os


sns.set_style('whitegrid')
warnings.filterwarnings('ignore')

In [19]:
df_res = pd.read_csv('gdf_svm_results.csv')
df_res.drop(columns=['Unnamed: 0'], inplace=True)
df_res.columns

Index(['C', 'coef0', 'f1', 'features', 'gamma', 'kappa', 'kernel', 'matthews',
       'precision', 'recall', 'roc_auc', 'stock', 'test_f1', 'test_kappa',
       'test_matthews', 'test_precision', 'test_recall', 'test_roc_auc',
       'train_f1', 'train_kappa', 'train_matthews', 'train_precision',
       'train_recall', 'train_roc_auc', 'r', 's', 'info'],
      dtype='object')

In [23]:
df_res[[c for c in df_res.columns if 'test' in c] + ['stock', 'features', 'info', 'kernel']].sort_values(
    by=['test_matthews'], ascending=False).groupby(['kernel', 'stock']).head(1)

,test_f1,test_kappa,test_matthews,test_precision,test_recall,test_roc_auc,stock,features,info,kernel
30,0.591207,0.125588,0.126779,0.555686,0.631579,0.562785,9265,pca_gdf_que_prev4,scaled,rbf
3,0.555108,0.117053,0.117071,0.560000,0.550300,0.558528,9265,pca_gdf_que_prev4,regular,sigmoid
23,0.604142,0.103959,0.106561,0.550701,0.669069,0.551778,9064,pca_gdf_que_prev7,regular,rbf
2,0.561884,0.080420,0.080569,0.546073,0.578637,0.540167,9064,pca_gdf_que_prev4,regular,sigmoid
13,0.440257,0.066667,0.070424,0.545726,0.368952,0.533251,9061,gdf_24-26_que_prev,regular,rbf
7,0.524232,0.070083,0.070116,0.532594,0.516129,0.535033,9061,pca_gdf_que5,regular,sigmoid


In [27]:
df_res[[c for c in df_res.columns if 'train' in c] + ['stock', 'features', 'info', 'kernel']].sort_values(
    by=['train_matthews'], ascending=False).groupby(['kernel', 'stock']).head(1)

,train_f1,train_kappa,train_matthews,train_precision,train_recall,train_roc_auc,stock,features,info,kernel
18,0.567047,0.128018,0.129550,0.563648,0.576675,0.563866,9265,pca_gdf_que8,regular,rbf
14,0.598567,0.126850,0.128258,0.565485,0.636643,0.563231,9064,pca_gdf_que7,regular,rbf
3,0.557673,0.113883,0.114008,0.555029,0.560917,0.556905,9265,pca_gdf_que_prev4,regular,sigmoid
25,0.489796,0.101111,0.103816,0.554815,0.441563,0.550224,9061,pca_gdf_que_prev3,scaled,rbf
4,0.534572,0.091287,0.091339,0.535643,0.533756,0.545616,9061,pca_gdf_que5,regular,sigmoid
2,0.562181,0.074645,0.074883,0.544943,0.581033,0.537265,9064,pca_gdf_que_prev4,regular,sigmoid
